In [1]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import numpy as np
import pandas as pd
from collections import defaultdict
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

conditional image captioning:

text = "a photography of"

inputs = processor(raw_image, text, return_tensors="pt")

unconditional image captioning:

inputs = processor(raw_image, return_tensors="pt")

out = model.generate(**inputs)

print(processor.decode(out[0], skip_special_tokens=True))

In [8]:
df = pd.read_csv(r'C:\Users\yotam\Desktop\naya\df_clean_after_pca')
df = df[df['rating']!=0]
df_isbn = df['isbn']
num_dict = {}
for i,j in enumerate(df_isbn.unique()):
    num_dict[j]=i
df['isbn_num'] = df['isbn'].apply(lambda txt: num_dict[txt])
df = df.reset_index(drop = True)
df_category = pd.DataFrame(df['img_l'].unique())
df_category = df_category.rename(columns={0:"img_l"})
df_category.iloc[1]['img_l']
df_category['cover_page'] = '9' #Default value, so that we can identify cases in which the model did not work (due to a problem loading the image)

'http://images.amazon.com/images/P/0374157065.01.LZZZZZZZ.jpg'

In [16]:
#example:
inputs = processor(raw_image, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

C:\Users\yotam\AppData\Roaming\Python\Python39\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a close up of a book cover with two women


In [20]:
df_category.loc[0,'cover_page']=processor.decode(out[0], skip_special_tokens=True)
#df_category = pd.read_csv(r'cover_page.csv')

In [4]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
for i in range(len(df_category)): 
    img_url = df_category.iloc[i]['img_l']

    try:
        response = requests.get(img_url, headers=headers, stream=True)

        response.raise_for_status()  # Check if the request was successful

        # Open the image from the response content (bytes) and convert to RGB mode
        raw_image = Image.open(response.raw).convert('RGB')

        # You can perform further processing with the `raw_image` object here

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the image from the URL: {e}")
        continue

    except Image.UnidentifiedImageError:
        print("Unable to identify the image file. Please check the URL or image format.")
        continue
    try:
        inputs = processor(raw_image, return_tensors="pt")
        out = model.generate(**inputs)
    except:
        continue
    df_category.loc[i,'cover_page']=processor.decode(out[0], skip_special_tokens=True)
    if i%200==0:
        print("reach",i)
        df_category.to_csv('cover_page.csv')

C:\Users\yotam\AppData\Roaming\Python\Python39\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


reach 25600
reach 25800
reach 26000
reach 26200
reach 26400
reach 26600
reach 26800
reach 27000
reach 27200
reach 27400
reach 27600
reach 27800
reach 28000
reach 28200
reach 28400
reach 28600
reach 28800
reach 29000
Error fetching the image from the URL: 404 Client Error: Not Found for url: http://images.amazon.com/images/P/0553566563.01.LZZZZZZZ.jpg
reach 29200
reach 29400
reach 29600
reach 29800
reach 30000
reach 30200
reach 30600
reach 30800


In [13]:
#example of the book covers as text:
for i in range(100):
    print(i, df.iloc[i]['cover_page'])

0 a close up of a book cover with two women
1 the story of the great influence of the pandemic in the world of war and the search
2 9
3 a poster for the movie night's below station street
4 a poster for a chicken soup for the woman's soul
5 a book cover of the testament
6 beloved by toni morrison
7 a close up of a newspaper with a bunch of headlines on it
8 a close up of a plate of food with a fork and knife
9 a cartoon book cover of tell me this isn't happening
10 a close up of a poster with a painting of a clock
11 airframe by michael crichton
12 a book cover of time line by michael crichton
13 a book cover of winter solstice
14 death in the clouds by agatha gitta
15 a book cover of the last of the breed
16 the cover of the book asmov prelude to foundation
17 out of the silent planet by c s lewis
18 a book cover of piercing the darkness
19 a book cover of the prophet
20 prague a novel by arthur phillips
21 chocolate jesus by stephen jarami
22 a close up of a book cover with a person 

In [24]:
df_category.to_csv('cover_page.csv')
df =df_category.copy()
df_category.drop(['Unnamed: 0.7','Unnamed: 0.6','Unnamed: 0.5','Unnamed: 0.4','Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0'],axis=1,inplace=True)

In this section, I process all the data, and do PCA for the embedding products of the book images, the book title, and the book summary together

In [5]:
from book_funcs import category_cleaner,df_cleaner, category_compliter, zero_droper, category_embedding, pca_embedding
df = pd.read_csv(r"C:\Users\yotam\Desktop\naya\Preprocessed_data.csv")
df_category = pd.read_csv(r'cover_page.csv')
df_isbn = df['isbn']
num_dict = {}
for i,j in enumerate(df_isbn.unique()):
    num_dict[j]=i
df['isbn_num'] = df['isbn'].apply(lambda txt: num_dict[txt])
df = category_cleaner(df)
df = df_cleaner(df)
df = category_compliter(df)
df = df.merge(df_category, how='left', on='img_l')
df = category_embedding(df)
df = category_embedding(df, col ='Summary')
df = category_embedding(df, col ='cover_page')
from sklearn.decomposition import PCA
df = df.drop(['Embedding_cat_x','Embedding_cat_y','Embedding_cat'],axis=1)
embeded_cols = [col for col in df.columns if ('Embedding' in col)]
not_embeded_cols = [col for col in df.columns if ('Embedding' not in col)]
df_embeding = df.drop(not_embeded_cols,axis=1)
df = df.drop(embeded_cols,axis=1)
#df.drop_duplicates(subset='isbn_num', keep="first",inplace=True)
pca = PCA()
df_embeding = pca.fit_transform(df_embeding)
df_embeding_df = pd.DataFrame(df_embeding)
df_embeding_df = df_embeding_df.drop([i for i in range(300,1152)],axis=1)
df = pd.merge(df.reset_index(), df_embeding_df.reset_index(), on = 'index', how = "left")
df = df.drop('index',axis =1)
df.to_csv('df_after_pca_with_cover.csv')

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

Batches:   0%|          | 0/649 [00:00<?, ?it/s]

Batches:   0%|          | 0/783 [00:00<?, ?it/s]

In this section I do PCA for the embedding products of the book cover image only.

In [44]:
from book_funcs import category_cleaner,df_cleaner, category_compliter, zero_droper, category_embedding, pca_embedding
df = pd.read_csv(r"C:\Users\yotam\Desktop\naya\Preprocessed_data.csv")
df_category = pd.read_csv(r'cover_page.csv')

In [45]:
df_isbn = df['isbn']
num_dict = {}
for i,j in enumerate(df_isbn.unique()):
    num_dict[j]=i
df['isbn_num'] = df['isbn'].apply(lambda txt: num_dict[txt])
df = category_cleaner(df)
df = df_cleaner(df)
df = category_compliter(df)
df = zero_droper(df)
df = df.merge(df_category, how='left', on='img_l')
#df = category_embedding(df)
#df = category_embedding(df, col ='Summary')
df = category_embedding(df, col ='cover_page')
from sklearn.decomposition import PCA
df = df.drop(['Embedding_cat'],axis=1)
embeded_cols = [col for col in df.columns if ('Embedding' in col)]
not_embeded_cols = [col for col in df.columns if ('Embedding' not in col)]
df_embeding = df.drop(not_embeded_cols,axis=1)
df = df.drop(embeded_cols,axis=1)

The original data frame shape:	(1031175, 20)
The data frame shape after bookk filtering:	(629588, 20)
The new data frame shape:	(581111, 20)


Batches:   0%|          | 0/783 [00:00<?, ?it/s]

the PCA

In [48]:
pca = PCA()
df_embeding2 = pca.fit_transform(df_embeding)
df_embeding_df = pd.DataFrame(df_embeding2)
df_embeding_df = df_embeding_df.drop([i for i in range(150,383)],axis=1)
df = pd.merge(df.reset_index(), df_embeding_df.reset_index(), on = 'index', how = "left")
df = df.drop('index',axis =1)
sum(pca.explained_variance_ratio_[0:150])

0.8897090882402773

In [58]:
df = df.drop([383],axis=1)
embeded_cols = [i for i in range(0,150)] +['isbn']
not_embeded_cols = [col for col in df.columns if (col not in embeded_cols)]
df = df.drop(not_embeded_cols,axis=1)
df =df.drop_duplicates()
df.to_csv('df_cover_pca.csv')